In [1]:
# Imports
import pymongo
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append('d:\\GitHub\\NextTrip\\flask_LocationModel\\config')

In [3]:
from db_config import DB_URL
import re

In [4]:
with open('/GitHub/NEXTTRIP/flask_LocationModel/config/db_config.py', 'r') as databaseUrlFile:
    DB_URL = databaseUrlFile.read()
DB_URL

match = re.search(r'"([^"]*)"', DB_URL)

if match:
    database_url = match.group(1)
else:
    print("No match found")

In [5]:
client = pymongo.MongoClient(database_url)

In [6]:
db = client['NextTripDB']
collection = db['rides_data']

In [7]:
# Retrieve the JSON data from MongoDB
data = list(collection.find({}))  
data_df = pd.DataFrame(data)
data_df.head()

,_id,Date/Time,Lat,Lon,__v
0,650db48407e33ccdd2398bf9,4/1/2014 0:11:00,40.7690,-73.9549,NaN
1,650db48407e33ccdd2398bfa,4/1/2014 0:17:00,40.7267,-74.0345,NaN
2,650db48407e33ccdd2398bfb,4/1/2014 0:21:00,40.7316,-73.9873,NaN
3,650db48407e33ccdd2398bfc,4/1/2014 0:28:00,40.7588,-73.9776,NaN
4,650db48407e33ccdd2398bfd,4/1/2014 0:33:00,40.7594,-73.9722,NaN


In [8]:
# Load the dataset
# data = pd.read_csv('/GitHub/NEXTTRIP/flask_LocationModel/data/uber-raw-data-apr14.csv')

In [9]:
df = pd.DataFrame(data_df)

In [10]:
# Preprocess the data
data_df['Date/Time'] = pd.to_datetime(data_df['Date/Time'])

In [11]:
# Feature Engineering
data_df['weekday'] = data_df['Date/Time'].dt.weekday
data_df['hour'] = data_df['Date/Time'].dt.hour

In [12]:
data_df.head()

,_id,Date/Time,Lat,Lon,__v,weekday,hour
0,650db48407e33ccdd2398bf9,2014-04-01 00:11:00,40.7690,-73.9549,NaN,1,0
1,650db48407e33ccdd2398bfa,2014-04-01 00:17:00,40.7267,-74.0345,NaN,1,0
2,650db48407e33ccdd2398bfb,2014-04-01 00:21:00,40.7316,-73.9873,NaN,1,0
3,650db48407e33ccdd2398bfc,2014-04-01 00:28:00,40.7588,-73.9776,NaN,1,0
4,650db48407e33ccdd2398bfd,2014-04-01 00:33:00,40.7594,-73.9722,NaN,1,0


In [13]:
#  Aggregating the data to count how many times each combination of latitude, longitude, weekday, and hour appears.
heatmap_data=df.groupby(['Lat','Lon','weekday','hour']).size().reset_index(name='weight')

In [14]:
heatmap_data.head()

,Lat,Lon,weekday,hour,weight
0,40.0729,-74.1638,1,14,1
1,40.1918,-74.6036,0,10,1
2,40.2026,-74.1224,2,16,1
3,40.2168,-74.5670,0,18,1
4,40.2259,-74.6235,2,21,1


In [15]:
heatmap_data_toSave = heatmap_data.to_dict(orient="records")

In [16]:
heatmap_data_toSave

[{'Lat': 40.0729, 'Lon': -74.1638, 'weekday': 1, 'hour': 14, 'weight': 1},
 {'Lat': 40.1918, 'Lon': -74.6036, 'weekday': 0, 'hour': 10, 'weight': 1},
 {'Lat': 40.2026, 'Lon': -74.1224, 'weekday': 2, 'hour': 16, 'weight': 1},
 {'Lat': 40.2168, 'Lon': -74.567, 'weekday': 0, 'hour': 18, 'weight': 1},
 {'Lat': 40.2259, 'Lon': -74.6235, 'weekday': 2, 'hour': 21, 'weight': 1},
 {'Lat': 40.2549, 'Lon': -74.7039, 'weekday': 3, 'hour': 18, 'weight': 1},
 {'Lat': 40.2643, 'Lon': -74.3553, 'weekday': 3, 'hour': 7, 'weight': 1},
 {'Lat': 40.2678, 'Lon': -74.544, 'weekday': 6, 'hour': 0, 'weight': 1},
 {'Lat': 40.2825, 'Lon': -74.1419, 'weekday': 4, 'hour': 21, 'weight': 1},
 {'Lat': 40.3082, 'Lon': -74.1148, 'weekday': 1, 'hour': 21, 'weight': 1},
 {'Lat': 40.3164, 'Lon': -74.625, 'weekday': 2, 'hour': 21, 'weight': 1},
 {'Lat': 40.3243, 'Lon': -74.6489, 'weekday': 0, 'hour': 21, 'weight': 1},
 {'Lat': 40.3268, 'Lon': -74.6406, 'weekday': 0, 'hour': 19, 'weight': 1},
 {'Lat': 40.3343, 'Lon': -74.6

In [17]:
db = client["insights"]
db.heatmap_data.insert_many(heatmap_data_toSave)

In [18]:
# csv_filepath = '/GitHub/NEXTTRIP/flask_LocationModel/data/heatmap_data.csv'
# heatmap_data.to_csv(csv_filepath, index=False)

In [19]:
hourcount_data=df.groupby(['hour']).size().reset_index(name='count')

In [20]:
hourcount_data.head()

,hour,count
0,0,11911
1,1,7769
2,2,4935
3,3,5040
4,4,6095


In [21]:
csv_filepath = '/GitHub/NEXTTRIP/flask_LocationModel/data/hourcount_data.csv'
hourcount_data.to_csv(csv_filepath, index=False)

In [22]:
daycount_data=df.groupby(['weekday']).size().reset_index(name='count')

In [23]:
daycount_data.head()

,weekday,count
0,0,60861
1,1,91185
2,2,108631
3,3,85067
4,4,90303


In [24]:
csv_filepath = '/GitHub/NEXTTRIP/flask_LocationModel/data/daycount_data.csv'
daycount_data.to_csv(csv_filepath, index=False)

In [25]:
dayHrcount_data=df.groupby(['weekday','hour']).size().reset_index(name='count')

In [26]:
dayHrcount_data.head()

,weekday,hour,count
0,0,0,518
1,0,1,261
2,0,2,238
3,0,3,571
4,0,4,1021


In [27]:
csv_filepath = '/GitHub/NEXTTRIP/flask_LocationModel/data/dayHrcount_data.csv'
dayHrcount_data.to_csv(csv_filepath, index=False)